<h1>Kapitel 7 Hypotesprövning</h1>

In [7]:
#importera paket och exempeldata 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as scs

#Importera Palmer Penguins
filepath = '../Data'
penguins = pd.read_csv(filepath)
penguins = penguins.dropna() # Plocka bort rader som innehåller NaN

FileNotFoundError: [Errno 2] No such file or directory: '../Statistiska_Metoder/Data/penguins.csv'

<h3>Exempel: Hypotestest för stickprovsmedelvärde med okänt sigma</h3>
<h5>Konstruera ett hypotestest som undersöker om längden på simfenorna för Gentoo-pingviner skiljer sig från längden på simfenor för alla pingviner (antag att populationsmedelvärdet $\mu_0$ = 216 mm). Använd en signifikansnivå på $\alpha = 0.05$.</h5>
Eftersom standardavvikelsen för populationen är okänd, kommer vi här använda oss av ett t-test.

In [3]:
from scipy.stats import t # Importera t-fördelningen ur SciPy

Vi börjar med att bestämma alternativhypotesen som:<br>
    $H_A: \mu \neq 216\,mm$<br>
Och sedan nollhypotesen som dess komplement:<br>
    $H_0: \mu = 216\,mm$

Sedan, beräknar vi teststatistikan för vår nollhypotes, d.v.s:

$t = \frac{\overline{X} - \mu_0}{s \,/\sqrt{n}}$
</h3>

<h5>Det här kan vi göra på lite olika sätt. Först kan vi göra det med klassiska beräkningsmetoder vi kan i NumPy.</h5>

In [5]:
gentoo = penguins[penguins['species'] == 'Gentoo'] # Subsetta data på species = Gentoo
sample = gentoo['flipper_length_mm'] # Välj ut kolonnen 'flipper_length_mm' som 'sample'.

xbar = sample.mean() # Beräkna stickprovsmedelvärde
mu = 216 # Sätt mu till värde för nollhypotesen
std = sample.std(ddof=1) # beräkna standardavvikelsen för stickprovet
n = len(sample) # beräkna storleken på stickprovet

statistic = (xbar - mu) / (std / np.sqrt(n)) # Beräkna värdet på test-statistiskan
print('Test statistic: ' +str(statistic)) # Printa test-statistikan

p_value = 1 - t.cdf(x=statistic, df=n-1) # Beräkna p-värdet för stick provet som 1 - F(x=statistic)
print('p-value: ' + str(2*p_value)) # Printa p-värdet (2x beräknat värde p.g.a. två-sidigt test.

NameError: name 'penguins' is not defined

<h5>Det finns också inbyggda funktioner för t.ex. t-test i SciPy, ni ska vi använda oss utav en av dem, vilket gör hela operationen mycket enklare.</h5>
Vi använder oss av den inbyggda funktionen ttest_1samp(), vilken beräknar t-test statistiska samt dess p-värde givet ett set data och ett populationsmedelvärde. Vi använder oss här av alternative='two-sided', vilket ger oss det dubbel-sidiga testet för att vårt stickprovsmedelvärde är skilt från populationsmedelvärdet.<br>
Dokumentationen till ttest_1samp() hittar vi <a href=https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_1samp.html>här</a>.

In [ ]:
result = scs.ttest_1samp(a=sample, popmean=mu, alternative='two-sided') # Genomför två-sidigt t-test m.h.a. ttest_1samp() i SciPy 

print(result) # Printa resultatet

Att använda den här funktionen blir väldigt praktiskt om vi t.ex. vill ändra på vår nollhypotes, låt säga att vi undrar om medelvärdet är större än 216 istället (en en-sidig nollhypotes). Vi ändrar då bara alternative till 'greater'. Vi vet sen tidigare att det här egentligen mest bara lägger till en faktor 1/2 på p-värdet. På samma sätt kan vi också ändra till 'less', vilket då beräknar utifrån nollhypotesen att stickprovsmedelvärdet är mindre än populationsmedelvärdet.

In [ ]:
result = scs.ttest_1samp(a=sample, popmean=mu, alternative='greater') # Genomför ensidigt t-test.

print(result) # Printa resultatet

Eftersom $p=0.043 < \alpha = 0.05$ kan vi med 95% konfidensgrad förkasta nollhypotesen att simfenelängden för Gentoo är samma som för alla pingviner. Testet stödjer alltså tesen att Gentoo-pingviner har en simfenelängd som skiljer sig från övriga pingviner.

<h3>Exempel: Hypotestest för proportioner</h3>
<h5>Konstruera ett hypotestest som undersöker om mindre än en tredjedel av alla Adeliepingviner återfinns på Biscoe Island. Använd ett signifikansvärde på $\alpha=0.1$</h5>
För proportionstest använder vi oss av standardnormalfördelningen Z, så vi börjar med att importera den från SciPy.

In [ ]:
from scipy.stats import norm # Importera normalfördelningen från SciPy

Sedan formulerar vi vår alternativhypotes som:<br>
    <h4>$H_A: p < 1/3$</h4>
Sedan formulerar vi vår nollhypotes som:<br>
    <h4>$H_0: p \geq 1/3$</h4>
    
Sedan beräknar vi test-statistikan för vår hypotes, som ges av:<br>

<h3>$Z = \frac{\hat{p}-p_0}{\sqrt{p_0(1-p_0)/n}}$</h3>

Först, med NumPy-metoder:

In [ ]:
adelie = penguins[penguins['species'] == 'Adelie']
adelie_biscoe = adelie[adelie['island'] == 'Biscoe']

p_sample = len(adelie_biscoe) / len(adelie)
p_pop = 1/3
n = len(adelie)

statistic = (p_sample - p_pop) / np.sqrt(p_pop*(1-p_pop)/n)
print('Test statistic: ' +str(statistic))

p_value = norm.cdf(statistic)
print('p-value: ' + str(p_value))

Låt oss nu göra det med en inbyggd funktion. Det finns inget Z-test för proportioner i SciPy, utan vi får importera ett från statsmodels istället.

In [ ]:
from statsmodels.stats import proportion

Här använder vi oss av testet proportions_ztest(), dokumentationen för den här funktionen hittar ni <a href=https://www.statsmodels.org/stable/generated/statsmodels.stats.proportion.proportions_ztest.html>här</a>. Vi använder oss av alternative='smaller', eftersom vår alternativhypotes är att p är <strong>mindre</strong> än 1/3. 

In [ ]:
result = proportion.proportions_ztest(count=len(adelie_biscoe), nobs=len(adelie), value=p_pop, prop_var=p_pop, alternative='smaller')
print(result)

Eftersom $p = 0.21 > \alpha=0.1$ kan vi inte förkasta nollhypotesen. Det betyder, att vi med en signifikansnivå på 0.1, inte kan säga att mindre än en tredjedel av alla Adelie-pingviner finns på Biscoeisland. 

<h3>Överkurs: Hypotestest för varians</h3>
Varianstest med $\chi^2$-fördelningen kommer inte att ingå i kursens examinationsmoment. Det är med i notebooken här som ett exempel för de som vill lära sig mer utöver det kursen går igenom. Att hypotesttesta med $\chi^2$-fördelningen tillför lite krångel, eftersom $\chi^2$-fördelningen inte är symmetrisk. Man kan alltså inte för ett två-sidigt test räkna på ett p-värde och sedan multiplicera det med en faktor två.


<h5>Konstruera ett hypotestest som undersöker om vikten hos Chinstrap-pingviner har mindre varians än vikten hos Adelie-pingviner (antag att variansen för Adelie-pingviner är känd som $\sigma^2_0 = 200000$). Använd en signifikansnivå på $\alpha = 0.01$.</h5>
Test på varians kan vi göra med hjälp av $\chi^2$-fördelningen. Den finns att importera från SciPy.

In [ ]:
from scipy.stats import chi2

Sedan formulerar vi vår alternativhypotes som:<br>
    <h4>$H_A: \sigma^2 < \sigma_0^2$</h4>
Och då blir vår nollhypotes:<br>
    <h4>$H_0: \sigma^2 \geq \sigma_0^2$</h4>
    
Sedan beräknar vi test-statistikan för vår hypotes, som ges av:<br>

<h3>$\chi^2 = \frac{(n-1)S^2}{\sigma_0^2}$</h3><br>
Här får vi enbart räkna på det med NumPy, då det inte finns någon väldistribuerad testfunktion för chi2-test av varians (vad jag vet!) :

In [ ]:
chinstrap = penguins[penguins['species'] == 'Chinstrap']

s_2 = chinstrap['body_mass_g'].var(ddof=1)
s_0 = 2e5
n = len(chinstrap)

statistic = (n-1) * s_2 / s_0
print('Test statistic: ' +str(statistic))

p_value = chi2.cdf(x=statistic, df=n-1)
print('p-value: ' + str(p_value))

Eftersom p-värdet $p=0.054 > \alpha=0.05$ så kan vi inte förkasta nollhypotesen. Vi kan alltså inte, med en signifikansnivå på 0.05, säga att variansen för Chinstrappingviners kroppsvikt är mindre än den hos Adeliepingviner (under antagande att den variansen är känd som 2e5).

<h3>Exempel på hur en funktion som utför hypotestest för varians skulle kunna skrivas:</h3>
Här använder vi en omvänd metod för att avgöra om nollhypotesen kan förkastas. Istället för att beräkna p-värdet vid ett beräknat värde på teststatistikan, så räknar vi ut vad teststatistiskan hade varit med ett givet p-värde. Detta görs med funktionen <code>ppf()</code>, som är inversen till <code>cdf()</code>. <code>ppf(alpha, n-1)</code> ger alltså värdet på test-statistikan $\chi^2$ vid en signifikansnivå $\alpha=0.05$. Se om du förstår hur nedanstående funktion fungerar!

In [ ]:
import numpy as np
from scipy.stats import chi2

def var_test(x, va0, direction = "lower", alpha = 0.05):
    n = len(x)
    Q = (n - 1) * np.var(x,ddof=1) / va0 
    if direction == "lower":
        q = chi2.ppf(alpha, n - 1)
        if Q <= q:
            return "H_0 rejected", Q
        else:
            return "H_0 not rejected", Q
    elif direction == "upper":
        q = chi2.ppf(1-alpha, n - 1)
        if Q >= q:
            return "H_0 rejected", Q
        else:
            return "H_0 not rejected", Q
    else:
        q1 = chi2.ppf(alpha/2, n - 1)
        q2 = chi2.ppf(1-(alpha/2), n - 1)
        if Q <= q1 or Q >= q2:
            return "H_0 rejected", Q
        else:
            return "H_0 not rejected", Q

In [ ]:
chinstrap_mass = chinstrap['body_mass_g']

var_test(chinstrap_mass, va0 = 2e5, direction='lower')